In [11]:
import pandas as pd
import json
import os
import re
import requests
print(os.getcwd())

c:\Users\Joshualevy\Documents\ioCapture\01_Database_Construction\papers


In [14]:
### PREAMBLE: LOADING LOCAL ENVIRONMENT VARIABLES (API KEYS) Accessing secure API keys
# TODO: need to generalize the placement of the env_keys.json to the top level of the directory (and modify .gitignore accordingly)
keys_json = json.load(open("../../00_Scraping/env_keys.json"))
scopus_key = keys_json['scopus_key']
aditya_key = keys_json['aditya_imperial_scopus_key']
thomas_key = keys_json['thomas_chicago_scopus_key']

req_headers = {
    'X-ELS-APIKey' : thomas_key
}

In [12]:
consultants_df = pd.read_csv('../consultants/nber_consultants.csv')
consultants_df = consultants_df.loc[consultants_df['consults_at_all'] == 1,:]
consultants_df = consultants_df.loc[consultants_df['scopus_id'].notna(),:]
consultants_df['scopus_id'] = consultants_df['scopus_id'].apply(lambda x: re.sub(r'\"', '', x))
consultants_df.head(5)


,name,nber_link,nber_program,personal_website_link,current_university_affiliation,nber_status,compass_lexecon,nera,analysis_group,cra,...,dicloses_cv_nber,firm_website_or_cv_nber,CV_consulting,University_Disclosure,NBER_disclosure,editor_positions,scopus_id,sex,TransparencyIdx,IdxNotes
0,John Asker,https://www.nber.org/people/john_asker?page=1&...,Industrial Organization,http://www.johnasker.com/,"University of California, Los Angeles",Research Associate,0,0,0.0,0,...,1,1,NaN,NaN,NaN,NaN,6603964906,1.0,1.0,Discloses in CV
1,Susan Athey,https://www.nber.org/people/susan_athey?page=1...,Industrial Organization,https://athey.people.stanford.edu/,Stanford University: GSB,Research Associate,0,0,0.0,0,...,1,1,NaN,NaN,NaN,NaN,6603691303,0.0,2.0,"Discloses in CV, discloses in NBER, consulting..."
2,Steven T. Berry,https://www.nber.org/people/steven_berry?page=...,Industrial Organization,https://www.steventberry.com/cv,Yale University,Research Associate,1,0,0.0,0,...,0,1,NaN,NaN,NaN,NaN,7201835962,1.0,0.0,NaN
3,Severin Borenstein,https://www.nber.org/people/severin_borenstein...,Industrial Organization,http://faculty.haas.berkeley.edu/borenste/?_ga...,"University of California, Berkeley: Haas",Research Associate,0,0,0.0,0,...,0,1,NaN,NaN,NaN,NaN,7004593708,1.0,1.0,NaN
4,Dennis W. Carlton,https://www.nber.org/people/dennis_carlton?pag...,Industrial Organization,https://www.chicagobooth.edu/faculty/directory...,University of Chicago: Booth,Research Associate,1,0,0.0,0,...,1,1,NaN,NaN,NaN,NaN,14419920600,1.0,2.0,"Discloses in CV, discloses in NBER, consulting..."


In [102]:
def compilePageOfResults(articles_list, results_list):
    fields_of_interest = ['dc:title', 'prism:publicationName', 'prism:issn', 'prism:volume', 'prism:issueIdentifier']

    for result in results_list:

        data_dict = {}
        for key in fields_of_interest:
            try: 
                data_dict[key] = result[key]
            except:
                data_dict[key] = None

        articles_list.append(data_dict)

    return articles_list

In [172]:
def getScholarsAllScopusPapers(obs):
    # print(obs)
    name = obs['name']
    sc_author_id = obs.scopus_id

    url = 'https://api.elsevier.com/content/search/scopus'
    response = requests.get(
        url = url,
        headers = req_headers,
        params = {
            'httpAccept': 'application/json',
            'query' : 'AU-ID({})'.format(sc_author_id),
            'date' : '{start_year}-{end_year}'.format(start_year=1990, end_year=2021),
            'view' : 'STANDARD'
        }
    ).json()


    results_count = int(response.get('search-results').get('opensearch:totalResults'))
    print('Number of articles identified for {}: {}'.format(name, results_count))

    results_list = response.get('search-results').get('entry')
    # print(len(results_list))
    articles_list = []
    if len(results_list) != results_count:
        # print(response.get('search-results'))
        number_collected_so_far = 0
        while number_collected_so_far < results_count:
            print('\tNumber collectd so far: {}'.format(number_collected_so_far))
            

            if number_collected_so_far == 0:
                # print('Compiling the first page collected on initial request')
                articles_list_one_page = compilePageOfResults(articles_list=articles_list, results_list=results_list)

                articles_list += articles_list_one_page
                next_page_url = response.get('search-results').get('link')[2].get('@href')
                # print('NEXT PAGE URL: {}'.format(next_page_url))

            else:
                response = requests.get(
                    url = next_page_url,
                    headers=req_headers,
                ).json()

                new_current_page_results_list = response.get('search-results').get('entry')
                next_page_url = response.get('search-results').get('link')[3].get('@href')
                # print(next_page_url)
                # print('\t Number of results in this page: {}'.format(len(new_current_page_results_list)))
                # print(response.get('search-results').get('link'))
                
                
                articles_list_one_page = compilePageOfResults(articles_list=[], results_list=new_current_page_results_list)
                articles_list += articles_list_one_page
                # print('NEXT PAGE URL: {}'.format(next_page_url))

            number_collected_so_far = len(articles_list)
            # print('end of while loop')



    else:
        articles_list = compilePageOfResults(articles_list=articles_list, results_list=results_list)
    
    temp_df = pd.DataFrame.from_dict(articles_list)
    temp_df['sc_author_id'] = sc_author_id
    return temp_df

In [ ]:
consultants_all_papers_df = pd.DataFrame()
for index, row in consultants_df.iterrows():
    temp_df = getScholarsAllScopusPapers(row)
    temp_df = temp_df.drop_duplicates()
    print('Length of {}_df: {}'.format(row['name'], len(temp_df)))

    consultants_all_papers_df = pd.concat([consultants_all_papers_df, temp_df], ignore_index=True)

consultants_all_papers_df

In [189]:
len(consultants_all_papers_df)
consultants_all_papers_df.columns

Index(['dc:title', 'prism:publicationName', 'prism:issn', 'prism:volume',
       'prism:issueIdentifier', 'sc_author_id'],
      dtype='object')

In [205]:
consultants_all_papers_df_cleaned = consultants_all_papers_df
consultants_names = consultants_df[['scopus_id', 'name']]


consultants_all_papers_df_cleaned = pd.merge(consultants_all_papers_df_cleaned, consultants_names, how='left', left_on='sc_author_id', right_on='scopus_id')
consultants_all_papers_df_cleaned = consultants_all_papers_df_cleaned.rename(columns={
    'dc:title' : 'Title',
    'prism:publicationName' : 'Journal',
    'prism:issn' : 'ISSN',
    'prism:volume' : 'Volume',
    'prism:issueIdentifier' : 'Issue',
    'name' : 'Name',
})
consultants_all_papers_df_cleaned = consultants_all_papers_df_cleaned.drop(columns=['sc_author_id', 'scopus_id'])
consultants_all_papers_df_cleaned = consultants_all_papers_df_cleaned[['Name', 'Title', 'Journal', 'Volume', 'Issue']]
consultants_all_papers_df_cleaned = consultants_all_papers_df_cleaned.sort_values(by=['Name', 'Journal', 'Volume', 'Issue'])
consultants_all_papers_df_cleaned

,Name,Title,Journal,Volume,Issue
2164,A. Mitchell Polinsky,Prison work programs in a model of deterrence,American Law and Economics Review,19,2
2172,A. Mitchell Polinsky,Public enforcement of law,Criminal Law and Economics,None,None
2174,A. Mitchell Polinsky,Fixed-Price Versus Spot-Price Contracts: A Stu...,Economic Analysis of the Law: Selected Readings,None,None
2163,A. Mitchell Polinsky,Strict liability vs. Negligence in a market se...,Economics and Liability for Environmental Prob...,None,None
2175,A. Mitchell Polinsky,Preface,Handbook of Law and Economics,1,None
...,...,...,...,...,...
3406,Yael Hochberg,Local overweighting and underperformance: Evid...,Review of Financial Studies,26,2
3405,Yael Hochberg,Informational holdup and performance persisten...,Review of Financial Studies,27,1
3404,Yael Hochberg,Specialization and Competition in the Venture ...,Review of Industrial Organization,46,4
3403,Yael Hochberg,Intangible but bankable,Science,348,6240


In [188]:
consultants_jouranals_agg_df = consultants_all_papers_df
journal_names_issns_df = consultants_all_papers_df[['prism:issn', 'prism:publicationName']].drop_duplicates()

consultants_journals_agg_df = consultants_jouranals_agg_df.groupby(by=['prism:issn']).size()
consultants_journals_agg_df = consultants_journals_agg_df.to_frame().reset_index()
consultants_journals_agg_df = pd.merge(consultants_journals_agg_df, journal_names_issns_df, how='left', on='prism:issn')
consultants_journals_agg_df = consultants_journals_agg_df.rename(columns={
    0 : 'Count',
    'prism:issn' : 'ISSN',
    'prism:publicationName' : 'Journal'
    })
consultants_journals_agg_df = consultants_journals_agg_df.drop_duplicates(subset=['ISSN'])
consultants_journals_agg_df = consultants_journals_agg_df.sort_values('Count', ascending=False)
consultants_journals_agg_df = consultants_all_papers_df[['Journal', 'Count', 'Index']]

,ISSN,Count,Journal
8,00028282,301,American Economic Review
239,0304405X,290,Journal of Financial Economics
77,00221082,206,Journal of Finance
295,08939454,170,Review of Financial Studies
118,00335533,107,Quarterly Journal of Economics
...,...,...,...
396,10964762,1,Thunderbird International Business Review
260,05738555,1,Contributions to Economic Analysis
258,03932729,1,International Spectator
399,10981616,1,Risk Management and Insurance Review
